In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import re
import tensorflow as tf
from keras.layers import LSTM , Bidirectional , Embedding, Dense , Input
from keras.preprocessing.text import Tokenizer
from keras.models import Model

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Awais\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Awais\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df=pd.read_csv('data\cyberbullying_tweets.csv')

In [3]:
df.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [4]:
df.columns= ['text','target']

In [5]:
df.head()

,text,target
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [6]:
df['target'].unique()

array(['not_cyberbullying', 'gender', 'religion', 'other_cyberbullying',
       'age', 'ethnicity'], dtype=object)

In [7]:
df.shape

(47692, 2)

In [8]:
df.isna().sum()

text      0
target    0
dtype: int64

In [9]:
df.duplicated().sum()

36

In [10]:
df.drop_duplicates(inplace=True)

# Text Preprocessing

In [11]:
stemmer=PorterStemmer()

In [12]:
from nltk.tokenize import word_tokenize , sent_tokenize

In [13]:
def preprocess(text):
    text = text.lower()
    text = word_tokenize(text)
    text = [word for word in text if not re.match(r'#\w+', word)]
    translator = str.maketrans('', '', string.punctuation)
    text = [word.translate(translator) for word in text]
    stop_words = set(stopwords.words('english'))
    text = [word for word in text if word not in stop_words]
    text=[stemmer.stem(word) for word in text]
    cleaned_text = " ".join(text)
    return cleaned_text


In [14]:
preprocess(df['text'][1])

' aussietv white   mkr  theblock  imacelebrityau  today  sunris  studio10  neighbour  wonderlandten  etc'

In [15]:
df['clean_text']=df['text'].apply(preprocess)

In [16]:
df.head()

,text,target,clean_text
0,"In other words #katandandre, your food was cra...",not_cyberbullying,word katandandr food crapilici mkr
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,aussietv white mkr theblock imacelebritya...
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,xochitlsuckkk classi whore red velvet cupcak
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,jasongio meh p thank head concern anoth an...
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,rudhoeenglish isi account pretend kurdish acc...


# Encoding the target categories

In [17]:
for i,value in enumerate(df['target'].unique()): 
    df['target'].replace({value:i},inplace=True)

In [18]:
df.head()

,text,target,clean_text
0,"In other words #katandandre, your food was cra...",0,word katandandr food crapilici mkr
1,Why is #aussietv so white? #MKR #theblock #ImA...,0,aussietv white mkr theblock imacelebritya...
2,@XochitlSuckkks a classy whore? Or more red ve...,0,xochitlsuckkk classi whore red velvet cupcak
3,"@Jason_Gio meh. :P thanks for the heads up, b...",0,jasongio meh p thank head concern anoth an...
4,@RudhoeEnglish This is an ISIS account pretend...,0,rudhoeenglish isi account pretend kurdish acc...


# Tokenizing the text

In [19]:
tokenizer=Tokenizer()

In [20]:
tokenizer.fit_on_texts(df['clean_text'].values)

In [21]:
sequences = tokenizer.texts_to_sequences(df['clean_text'].values)

In [22]:
len(tokenizer.word_index)

53268

In [23]:
sequences[:2] # Preview of sequences

[[99, 2158, 638, 18494, 35],
 [18495, 36, 35, 12413, 7350, 168, 12414, 18496, 6015, 18497, 236]]

# Padding the sequences

In [24]:
from keras.preprocessing.sequence import pad_sequences

In [25]:
max_len=40

In [26]:
padded_sequences=pad_sequences(sequences, maxlen=max_len, padding='post')

In [27]:
padded_sequences.shape

(47656, 40)

In [28]:
from keras.utils import to_categorical

In [29]:
x=padded_sequences
y=to_categorical(df['target'].values)

In [30]:
print(f'x has shape: {x.shape} and y has shape :{y.shape}')

x has shape: (47656, 40) and y has shape :(47656, 6)


In [31]:
from sklearn.model_selection import train_test_split

In [32]:
x_train, x_test, y_train, y_test =  train_test_split(x, y , test_size=0.1, random_state=42)

In [33]:
print(f'Traning has samples: {x_train.shape[0]} , Testing has samples: {x_test.shape[0]}')

Traning has samples: 42890 , Testing has samples: 4766


# Model Preparation

In [34]:
def build_model(embedding_dim , latent_dim , vocab_size , max_len, target_values):
    inp=Input(max_len, )
    emb=Embedding(vocab_size, embedding_dim , input_length=max_len)(inp)
    lstm_1=Bidirectional(LSTM(latent_dim, return_sequences=True))(emb)
    lstm_2=Bidirectional(LSTM(latent_dim))(lstm_1)
    out=Dense(target_values, activation='softmax')(lstm_2)
    model=Model(inp , out)
    return model

In [35]:
!pip install wandb

In [36]:
import wandb
wandb.login(key="ba7560e300da0402f0be06a309b33c370c62acfe")
wandb.init(project='Cyberbullying_Classification_v2')

wandb: Currently logged in as: awaistahseen009. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Awais\.netrc


In [37]:
wandb.config={
'batch_size':256,
'epochs':30,
'latent_dim':150,
'embedding_dim':300,
'optimizer':'rmsprop',
}
configuration=wandb.config
from wandb.keras import WandbCallback

In [38]:
vocab_size=len(tokenizer.word_index)
target_values=len(df['target'].unique())

In [39]:
target_values

6

In [40]:
model=build_model(configuration['embedding_dim'], configuration['latent_dim'], vocab_size+1 ,max_len, target_values )

In [41]:
model.compile(optimizer=configuration['optimizer'], loss='categorical_crossentropy' , metrics=['accuracy'] )

In [42]:
from tensorflow.keras.callbacks import EarlyStopping
early_s = EarlyStopping(monitor='val_loss',verbose=1,patience=10)

In [43]:
model.fit(x_train, y_train,
          epochs=configuration['epochs'] , 
          batch_size=configuration['batch_size'],
          validation_split=0.2,
          callbacks=[early_s, WandbCallback()]
         )

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/30


135/135 [==============================] - ETA: 0s - loss: 0.8699 - accuracy: 0.6315

C:\Users\Awais\anaconda3\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


INFO:tensorflow:Assets written to: C:\Users\Awais\Desktop\Machine_learning_ETEP\Cyberbullying_Classification\cyber-bullying-classification\wandb\run-20240204_124259-5vfpyhdr\files\model-best\assets


INFO:tensorflow:Assets written to: C:\Users\Awais\Desktop\Machine_learning_ETEP\Cyberbullying_Classification\cyber-bullying-classification\wandb\run-20240204_124259-5vfpyhdr\files\model-best\assets
wandb: Adding directory to artifact (C:\Users\Awais\Desktop\Machine_learning_ETEP\Cyberbullying_Classification\cyber-bullying-classification\wandb\run-20240204_124259-5vfpyhdr\files\model-best)... Done. 0.6s


135/135 [==============================] - 175s 1s/step - loss: 0.8699 - accuracy: 0.6315 - val_loss: 0.6792 - val_accuracy: 0.7427
Epoch 2/30
135/135 [==============================] - ETA: 0s - loss: 0.4326 - accuracy: 0.8169

C:\Users\Awais\anaconda3\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


INFO:tensorflow:Assets written to: C:\Users\Awais\Desktop\Machine_learning_ETEP\Cyberbullying_Classification\cyber-bullying-classification\wandb\run-20240204_124259-5vfpyhdr\files\model-best\assets


INFO:tensorflow:Assets written to: C:\Users\Awais\Desktop\Machine_learning_ETEP\Cyberbullying_Classification\cyber-bullying-classification\wandb\run-20240204_124259-5vfpyhdr\files\model-best\assets
wandb: Adding directory to artifact (C:\Users\Awais\Desktop\Machine_learning_ETEP\Cyberbullying_Classification\cyber-bullying-classification\wandb\run-20240204_124259-5vfpyhdr\files\model-best)... Done. 0.4s


135/135 [==============================] - 149s 1s/step - loss: 0.4326 - accuracy: 0.8169 - val_loss: 0.4351 - val_accuracy: 0.8071
Epoch 3/30
135/135 [==============================] - 119s 880ms/step - loss: 0.3443 - accuracy: 0.8612 - val_loss: 0.5497 - val_accuracy: 0.7940
Epoch 4/30
135/135 [==============================] - 116s 861ms/step - loss: 0.2886 - accuracy: 0.8861 - val_loss: 0.5583 - val_accuracy: 0.7967
Epoch 5/30
135/135 [==============================] - 116s 864ms/step - loss: 0.2461 - accuracy: 0.9048 - val_loss: 0.4790 - val_accuracy: 0.8268
Epoch 6/30
135/135 [==============================] - 117s 867ms/step - loss: 0.2101 - accuracy: 0.9209 - val_loss: 0.6033 - val_accuracy: 0.8122
Epoch 7/30
135/135 [==============================] - 118s 873ms/step - loss: 0.1821 - accuracy: 0.9302 - val_loss: 0.6194 - val_accuracy: 0.7940
Epoch 8/30
135/135 [==============================] - 115s 853ms/step - loss: 0.1624 - accuracy: 0.9367 - val_loss: 0.5907 - val_accuracy:

In [44]:
evaluation_result = model.evaluate(x_test, y_test)

# Printing the evaluation results
print("Evaluation Loss:", evaluation_result[0])
print("Evaluation Accuracy:", evaluation_result[1])

149/149 [==============================] - 6s 40ms/step - loss: 0.6704 - accuracy: 0.8168
Evaluation Loss: 0.6704307794570923
Evaluation Accuracy: 0.8168275356292725


In [45]:
model.save('cyber_model.keras')

In [46]:
wandb.finish()

accuracy,▁▅▆▇▇▇██████
epoch,▁▂▂▃▄▄▅▅▆▇▇█
loss,█▄▃▃▂▂▂▁▁▁▁▁
val_accuracy,▁▆▅▅█▇▅▆▄▅▃▇
val_loss,▅▁▃▃▂▄▄▄▇▄█▆
accuracy,0.95255
best_epoch,1
best_val_loss,0.43506
epoch,11
loss,0.11034
val_accuracy,0.8101
